# Project1 PNLF Postproduction <a class="tocSkip">
    
After running the production notebook, this notebook can be used to create shared plots for the galaxies and LaTeX output tables

## Preparation
     
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from pnlf.packages import *

from pnlf.constants import tab10, single_column, two_column

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

we use the `logging` module to handle informations and warnings (this does not always work as expected in jupyter notebooks).

In [ ]:
logging.basicConfig(stream=sys.stdout,format='%(levelname)s: %(message)s',level=logging.INFO)
logger = logging.getLogger(__name__)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [ ]:
# first we need to specify the path to the raw data
basedir = Path('..')
data_raw = Path('a:')
data_ext = Path('a:')

with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)
    
results = ascii.read(basedir/'data'/'interim'/ 'results.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results.add_index('name')
#results.rename_columns(['(m-M)','err+(m-M)','err-(m-M)','mu_SNR','mu_SNR+','mu_SNR-'],['dis','dis_plus','dis_minus','dis_SNR','dis_SNR_plus','dis_SNR_minus'])

with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)    
    
sample_table = ascii.read(basedir/'data'/'interim'/'sample.txt')
sample_table.add_index('Name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])

In [ ]:
for row in results[np.abs(results['(m-M)']-results['mu_SNR'])>0.1]:
    print(row['name'], row['(m-M)']-row['mu_SNR'])

## LaTeX sample table

In [ ]:
sample_old = ascii.read(basedir/'data'/'interim'/'sample.txt')
sample_old.add_index('Name')
sample_old['SkyCoord'] = SkyCoord(sample_old['R.A.'],sample_old['Dec.'])

In [ ]:
sub = sample_table[sample_table['survey_muse_status']!='not_in_survey']

sub[['name', 'agn_veron_y_n',
 'agn_veron_class',
 'agn_milliquas_y_n',
 'agn_milliquas_class_pQSO']]

In [ ]:
from pnlf.auxiliary import search_table, parsec_to_mu

def area(dic):
    '''Calculate the area in an image
    
    from the number of pixels and the distance,
    corrected for inclination
    '''
    
    size_of_pixel = 0.2*u.arcsec

    distance = Distance(distmod=dic['mu'])
    pixel_area = (size_of_pixel/u.arcsec * distance/206265)**2
        
    return pixel_area.to(u.kpc**2) *dic['Npixel'] / np.cos(dic['inclination']*u.deg)


filename = basedir / 'data' / 'external' / 'phangs_sample_table_v1p6.fits'
with fits.open(filename) as hdul:
    sample_table = Table(hdul[1].data)

#galaxies = sample_table[sample_table['HAS_MUSE']==1]['NAME']
galaxies = sample_table[sample_table['survey_muse_status']!='not_in_survey']['name']

latexdict = {'tabletype': 'table*',
'header_start': '\\toprule\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'Galaxy sample',
'units': {'R.A.':'(J2000)','Dec.':'(J2000)','Inclination':'deg','Distance':'$\si{\mega\parsec}$',
          'Survey Area':'$\si{\square\kilo\parsec}$'},
'preamble': '\\centering',
'tablefoot': f'\\label{{tbl:sample}}'
            }
 
sample_dict = {
'Name': [],
'Type':[],
'R.A.': [],
'Dec.': [],
'Distance': [],
'Distance_err': [],
'(m-M)':[],
'err(m-M)':[],
'Inclination': [],
'posang': [],
'r25': [],
'12+log(O/H)': [],
'mass': [],
'SFR': [],
'Survey Area': [],
'E(B-V)': [],
'AO' : []
}

    
for name in galaxies:
    stbl = search_table(sample_table,name)
    d = Distance(stbl["dist"][0]*u.Mpc).distmod.value
    d_unc = 5/(stbl["dist"][0]*np.log(10)) * stbl["dist_unc"][0]

    sample_dict['Type'].append(stbl['morph_string'])    
    sample_dict['Name'].append(name.upper())
    sample_dict['R.A.'].append(stbl['orient_ra'])
    sample_dict['Dec.'].append(stbl['orient_dec'])
    sample_dict['Distance'].append(stbl["dist"][0])
    sample_dict['Distance_err'].append(stbl["dist_unc"][0])
    sample_dict['(m-M)'].append(0)
    sample_dict['err(m-M)'].append(0)    
    sample_dict['Inclination'].append(stbl['orient_incl'])
    sample_dict['posang'].append(stbl['orient_posang'])
    sample_dict['r25'].append((stbl['size_r25']*u.arcsec).to(u.arcmin).value)
    sample_dict['12+log(O/H)'].append(0.00)
    sample_dict['mass'].append(stbl['props_mstar'])
    sample_dict['SFR'].append(stbl['props_sfr'])
    sample_dict['E(B-V)'].append(sample_old.loc[name.upper()]['E(B-V)'])
    if parameters[name.upper()]['power_index'] == 2.3:
        sample_dict['AO'].append('\checkmark')
    else:
        sample_dict['AO'].append('')

    try:
        sample_dict['Survey Area'].append(f'{area(parameters[name.upper()]).value:.1f}')
    except:
        sample_dict['Survey Area'].append('NaN')
    '''
    sample_dict['Name'].append(name)
    sample_dict['R.A.'].append(stbl['ORIENT_RA'])
    sample_dict['Dec.'].append(stbl['ORIENT_DEC'])
    sample_dict['Distance'].append(stbl['DIST'])
    sample_dict['Inclination'].append(stbl['ORIENT_INCL'])
    sample_dict['$\log_{10}(M_*)$'].append(stbl['MSTAR_LOGMSTAR'])
    sample_dict['$\log_{10}($SFR$)$'].append(stbl['SFR_LOGSFR'])
    '''

sample = Table(sample_dict)
sample.add_index('Name')
sample['mass'] = np.log10(sample['mass'])
sample['SFR'] = np.log10(sample['SFR'])
err_up = 10 ** (np.log10(sample['Distance']) + sample['Distance_err']) - sample['Distance']
err_down = sample['Distance'] - 10 ** (np.log10(sample['Distance']) - sample['Distance_err'])

sample['Distance_err'] = err_up
sample['(m-M)'] = Distance(sample['Distance']*u.Mpc).distmod.value
sample['err(m-M)'] = 5/np.log(10)*sample['Distance_err'] / sample['Distance']

sample.loc['NGC1433']['(m-M)'] = 29.78
sample.loc['NGC1433']['err(m-M)'] = 0.49
sample.loc['NGC1512']['(m-M)'] = 30.33
sample.loc['NGC1512']['err(m-M)'] = 0.40

sample['Distance'].info.format = '%.2f' 
sample['Distance_err'].info.format = '%.2f' 
sample['(m-M)'].info.format = '%.2f' 
sample['err(m-M)'].info.format = '%.2f' 
sample['r25'].info.format = '%.2f' 
sample['mass'].info.format = '%.2f' 
sample['SFR'].info.format = '%.2f' 
coord = SkyCoord(sample['R.A.']*u.degree,sample['Dec.']*u.degree)
sample['R.A.'], sample['Dec.'] = zip(*[x[0].split(' ') for x in coord.to_string(style='hmsdms',precision=2)])
#sample.add_column('',index=1,name='Type')

sample

In [ ]:
with open(basedir / 'data' / 'interim' / 'sample.txt','w',newline='\n') as f:
    ascii.write(sample,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')
    
sample.add_column([f'\galaxyname{{NGC}}{{{row["Name"][-4:]}}}' for row in sample],index=0,name='galname')

sample.remove_columns(['mass','Survey Area','Distance','Distance_err','SFR'])
#sample.rename_columns(['mass','SFR'],['$\log_{10}(M_*/\si{\Msun})$','$\log_{10}($SFR$/\si{\Msun \per \year})$'])
with open(basedir / 'data' / 'interim' /'sample.tex','w',newline='\n') as f:
    ascii.write(sample,f,Writer=ascii.Latex, latexdict=latexdict,overwrite=True,exclude_names=['Name'])
    
#sample

## LaTeX result table

this uses the result table and prints out a LaTeX table (only the data part) that can be used in the final document. In another step, we merge the individual catalogues for PN and SNR identifications. 

In [ ]:
results['dis_Mpc'] = Distance(distmod=results['(m-M)']).to(u.Mpc)
results['dis_Mpc_plus'] = (Distance(distmod=results['(m-M)']+results['err+(m-M)']) - results['dis_Mpc']).to(u.Mpc)
results['dis_Mpc_minus'] = (results['dis_Mpc']-Distance(distmod=results['(m-M)']-results['err-(m-M)'])).to(u.Mpc)

#results['dis_SNR_Mpc'] = (Distance(distmod=results['dis_SNR'])).to(u.Mpc)
#results['dis_SNR_Mpc_plus'] = (Distance(distmod=results['dis_SNR']+results['dis_SNR_plus']) - results['dis_SNR_Mpc']).to(u.Mpc)
#results['dis_SNR_Mpc_minus'] = (results['dis_SNR_Mpc']-Distance(distmod=results['dis_SNR']-results['dis_SNR_minus'])).to(u.Mpc)

In [ ]:
# save to file
for col in ['(m-M)','err+(m-M)','err-(m-M)','dis_Mpc','dis_Mpc_plus','dis_Mpc_minus']:
    results[col].info.format = '%.2f'

distance_modulus = []
distance_parsec = []
for row in results:
    distance_modulus.append(f'{row["(m-M)"]:.2f} + {row["err-(m-M)"]:.2f} - {row["err+(m-M)"]:.2f}')
    distance_parsec.append(f'{row["dis_Mpc"]:.2f} + {row["dis_Mpc_plus"]:.2f} - {row["dis_Mpc_plus"]:.2f}')
results['mu'] = distance_modulus
results['d/Mpc'] = distance_parsec
                           
filename = basedir / 'data' / 'interim' / f'distances.txt'
with open(filename,'w',newline='\n') as f:
    ascii.write(results[['name','mu','d/Mpc']],
                f,format='fixed_width',delimiter='\t',overwrite=True)

In [ ]:
tbl_out = ''
results.sort('name')
for row in results:
    tbl_out += f'{row["name"]} & {row["N_PN"]} & {row["N_SNR"]} ({row["N_SNRorPN"]}) '
    tbl_out += f'& $\\uncertainty{{{row["(m-M)"]:.2f}}}{{{row["err+(m-M)"]:.2f}}}{{{row["err-(m-M)"]:.2f}}}$ '
    tbl_out += f'& $\\uncertainty{{{row["dis_Mpc"]:.2f}}}{{{row["dis_Mpc_plus"]:.2f}}}{{{row["dis_Mpc_minus"]:.2f}}}$ '
    tbl_out += f'& {row["alpha"]:.2f}\\\\\n'
    #tbl_out += f'& $\\uncertainty{{{row["dis_SNR"]:.2f}}}{{{row["dis_SNR_plus"]:.2f}}}{{{row["dis_SNR_minus"]:.2f}}}$ '
    #tbl_out += f'& $\\uncertainty{{{row["dis_SNR_Mpc"]:.2f}}}{{{row["dis_SNR_Mpc_plus"]:.2f}}}{{{row["dis_SNR_Mpc_minus"]:.2f}}}$\\\\\n'
    
    
print(tbl_out)    

In [ ]:
tbl_out = ''
results.sort('name')
for row in results:
    print(f'\\setvalue{{mag/{row["name"]} =\\SI{{\\uncertainty{{{row["(m-M)"]:.2f}}}{{{row["err+(m-M)"]:.2f}}}{{{row["err-(m-M)"]:.2f}}}}}{{\\mag}}}}')
print(' ')
for row in results:
    print(f'\\setvalue{{Mpc/{row["name"]} =\\SI{{\\uncertainty{{{row["dis_Mpc"]:.2f}}}{{{row["dis_Mpc_plus"]:.2f}}}{{{row["dis_Mpc_minus"]:.2f}}}}}{{\\mega\\parsec}}}}')
    

## Combine Catalogues to single file

In [ ]:
tbl_lst = []
ID_lst = []
notes_lst = []
for name in results['name']:
    tmp = ascii.read(basedir/'data'/'catalogues'/f'{name}_PN_candidates.txt',format='fixed_width_two_line')
    ID_lst += list(tmp['ID'])
    notes_lst += list(tmp['notes'])
    
    del tmp['ID']
    del tmp['notes']
    
    tbl_lst.append(tmp)
    
tbl = vstack(tbl_lst)
tbl.add_column(ID_lst,index=1,name='ID')
tbl.add_column(notes_lst,index=1,name='notes')


with open(basedir/'data'/'catalogues'/'PN_candidates.txt','w',newline='\n') as f:
    ascii.write(tbl,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')

In [ ]:
tbl_lst = []
for name in results['name']:
    tmp = ascii.read(basedir/'data'/'catalogues'/f'{name}_nebulae.txt',format='fixed_width_two_line')
    tmp['gal_name'] = name
    tbl_lst.append(tmp)
catalogue = vstack(tbl_lst)
catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['overluminous'] = catalogue['overluminous'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue['RA'],catalogue['DEC'] = zip(*[x.split(' ') for x in catalogue['SkyCoord'].to_string(style='hmsdms',precision=2)])

In [ ]:
row

In [ ]:
overluminous = catalogue[catalogue['overluminous']]

for row in overluminous[['gal_name','id','type','RA','DEC','mOIII','dmOIII']]:
    print('{} & {} & {} & {} & {} & {} & {} \\\\'.format(*row))
    
overluminous[['gal_name','id','type','RA','DEC','mOIII','dmOIII']]

## Single PNLF

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, pnlf, cdf
from pnlf.plot.pnlf import plot_pnlf
from pnlf.auxiliary import mu_to_parsec
from scipy.stats import kstest

name = 'NGC1672'

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)

binsize = 0.4 #parameters[name]['binsize']
completeness_limit = parameters[name]['completeness_limit']
mu = parameters[name]['mu']

# ----------------------------------------------------------------------
# fit the data
# ----------------------------------------------------------------------
'''
data = catalogue['mOIII'][(catalogue['type']=='PN') & (catalogue['mOIII']<completeness_limit)]
err = catalogue['dmOIII'][(catalogue['type']=='PN') & (catalogue['mOIII']<completeness_limit)]

fitter = MaximumLikelihood1D(pnlf,data,err=err,mhigh=completeness_limit,Mmax=-4.47)
mu,mu_p,mu_m = fitter([29])

d,(dp,dm)=mu_to_parsec(mu,[mu_p,mu_m])
print('{:.2f} + {:.2f} - {:.2f}'.format(mu,mu_p,mu_m))
print('{:.2f} + {:.2f} - {:.2f}'.format(d,dp,dm))

ks,pv = kstest(data,cdf,args=(mu,completeness_limit))
print(f'{name}: statistic={ks:.3f}, pvalue={pv:.3f}')
'''

# ----------------------------------------------------------------------
#plot PNLF
# ----------------------------------------------------------------------
filename = None #basedir / 'reports' / f'{galaxy.name}' / f'{galaxy.name}_PNLF'
axes = plot_pnlf(catalogue['mOIII'][(catalogue['type']=='PN')],mu,completeness_limit,
                 binsize=binsize,mhigh=29,filename=filename,color=tab10[0])
plt.show()

## Combined PNLF

In [ ]:
# here we define possible lists of objects to plot
DR2  = ['NGC1300','NGC1385','NGC1433','NGC4303','NGC4321','NGC7496']
good = ['IC5332','NGC0628','NGC1566','NGC3351','NGC3627','NGC5068']
PHANGS_sample = ['IC5332','NGC1087','NGC1300','NGC1365','NGC1385','NGC1433', 'NGC1512','NGC1566','NGC1672',
                 'NGC2835','NGC3351','NGC3627','NGC4254','NGC4303','NGC4321','NGC4535','NGC5068','NGC7496']

In [ ]:
from pnlf.plot.pnlf import _plot_pnlf

names = results['name']
nrows = 5
ncols = 4
filename = basedir / 'reports' / f'all_galaxies_PNLF'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
if nrows*ncols<len(names):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
Mmax = -4.47
    
# loop over the galaxies we want to plot
for name in names:  
    # read in the data
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
        catalogue['overluminous'] = catalogue['overluminous'].astype(bool)

    else:
        print(f'no catalogue for {name}')
        continue
        
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0
        
    # pre-process the data for the plot and read additional parameters
    data = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude']) & (~catalogue['overluminous'])]['mOIII']
    mu = results.loc[name]['(m-M)']
    completeness = parameters[name]['completeness_limit']
    cut = parameters[name].get('cut',0)
    if cut>0:
        print(f'warning: using cut={cut} for {name}')
    data = data[data>cut]
    binsize = parameters[name]['binsize']
    mlow = Mmax+mu
    mhigh = 28.5

    ax=_plot_pnlf(data,mu,completeness,binsize=binsize,mhigh=mhigh,ax=ax,ms=3)
    ax.text(0.4,0.08,f'{name}', transform=ax.transAxes,fontsize=7)

    #ax.set_xlim([mu-5,completeness+0.5])
    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'N')
    #ax.set_title(name)
    #ax.set(xlim=[24,28.5])
    
axes[3,3].set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
ax = next(axes_iter)
#ax.remove()
h,l = fig.axes[0].get_legend_handles_labels()
ax.axis('off')
ax.legend(h,l,fontsize=7,loc='center left',frameon=False)

plt.subplots_adjust(wspace=0.15, hspace=0.15)
#plt.tight_layout()
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

## Combined cumulative PNLF

In [ ]:
from pnlf.plot.pnlf import _plot_cum_pnlf
from pnlf.analyse import cdf
from scipy.stats import kstest

names = results['name']
nrows = 5
ncols = 4
filename = basedir / 'reports' / f'all_galaxies_PNLF_cum'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
if nrows*ncols<len(names):
    raise ValueError('not enough subplots for selected objects')
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

binsize=0.1
Mmax = -4.47
color = 'tab:red'

for name in names:  
        
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
        catalogue['overluminous'] = catalogue['overluminous'].astype(bool)
    else:
        print(f'no catalogue for {name}')
        continue
    
    # get the next axis
    ax = next(axes_iter)
    # find current position on the grid
    i, j = np.where(axes == ax)
    i,j=i[0],j[0]
    
    data = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude']) & (~catalogue['overluminous'])]['mOIII']
    mu = results.loc[name]['(m-M)']
    completeness = parameters[name]['completeness_limit']
    cut = parameters[name].get('cut',0)
    if cut>0:
        print(f'warning: using cut={cut} for {name}')
    data = data[data>cut]

    ks,pv = kstest(data[data<completeness],cdf,args=(mu,completeness))

    ax=_plot_cum_pnlf(data,mu,completeness,ax=ax,binsize=None)
    ax.text(0.55,0.08,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.1,0.78,f'$p$-value$={pv:.2f}$',transform=ax.transAxes,fontsize=7)
    ax.text(0.1,0.88,f'$D_{{max}}={ks:.3f}$', transform=ax.transAxes,fontsize=7)

    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'Cumulative N')
    
    #ax.set_title(name)
axes[3,3].set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
ax = next(axes_iter)
#ax.remove()

h,l = fig.axes[0].get_legend_handles_labels()
ax.axis('off')
ax.legend(h,l,fontsize=7,loc='center left',frameon=False)
    
plt.subplots_adjust(wspace=0.22, hspace=0.22)
#plt.tight_layout()
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

## Combine line diagnostics

In [ ]:
from pnlf.analyse import PNLF

names = result['name']
style = {'SNR':{"marker":'o',"ms":3,"mfc":'None',"mec":tab10[0],'ls':'none','ecolor':tab10[0]},
         'SNRorPN':{"marker":'o',"ms":4,"mfc":'white',"mec":'tab:green','ls':'none','ecolor':'tab:green'},
         'HII':{"marker":'+',"ms":3,"mec":tab10[1],'ls':'none'},
         'PN':{"marker":'o',"ms":2,"mfc":'black','mec':'black','ls':'none','ecolor':'black'}
        }
Mmax=-4.47
color = 'tab:red'

# define the figure with the number of subplots
nrows = 2
ncols = 3
width = two_column
fig, axes_arr = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes = iter(axes_arr.flatten())


names = ['IC5332','NGC0628','NGC1566','NGC3351','NGC3627','NGC5068']

# loop over the galaxies we want to plot
for name in names:  
       
    # read in the data
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
        for col in catalogue.columns:
            if col.endswith('detection'):
                catalogue[col] = catalogue[col]=='True'
    else:
        print(f'no catalogue for {name}')
        continue
        
    # get the next axis
    ax = next(axes)
    # find current position on the grid
    i, j = np.where(axes_arr == ax)
    i,j=i[0],j[0]
    
    # pre-process the data for the plot and read additional parameters
    mu = result.loc[name]['dis']
    completeness = parameters[name]['completeness_limit']
    
    # draw line that we use seperate PN from HII
    MOIII = np.linspace(-5,1)
    OIII_Ha = 10**(-0.37*(MOIII)-1.16)
    ax.plot(MOIII,OIII_Ha,c='black',lw=0.6)
    ax.axhline(10**4)

    if completeness:
        ax.axvline(completeness-mu,ls='--',c='grey',lw=0.5)
    ax.axvline(Mmax,ls='--',c='grey',lw=0.5)

    for t in ['HII','PN','SNR']:
        tbl = catalogue[catalogue['type']==t]        
        ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),**style[t],label=t) 

        if t=='PN':
            # indicate for which PN we don't have a detection in HA6562
            tbl = tbl[~tbl['HA6562_detection']]
            ax.errorbar(tbl['mOIII']-mu,1.11*tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),
                         marker=r'$\uparrow$',ms=4,mec='black',ls='none') 
        if t=='SNR':
            #tbl = tbl[tbl['SNRorPN']] 
            ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']), marker='x',ms=2,mec=tab10[0],ls='none') 
   
    # objects that were rejeceted by eye
    tbl = catalogue[catalogue['exclude']]
    ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),marker='o',ms=3,ls='none',color='tab:green',label='rejected') 
    
    
    # configure axes 
    ax.set(xlim=[-5,np.ceil(completeness-mu)],
           ylim=[0.03,200],
           yscale='log')
    
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.16g}'.format(y)))

    axt = ax.twiny()
    xlim1,xlim2 = ax.get_xlim()
    axt.set_xticks(np.arange(np.ceil(xlim1+mu),np.floor(xlim2+mu)+1),minor=False)
    axt.set(xlim   = [xlim1+mu,xlim2+mu])
    
    if i==0:
        axt.set(xlabel = r'$m_{\mathrm{[OIII]}}$')
    
    if i==nrows-1:
        ax.set(xlabel=r'$M_{\mathrm{[OIII]}}$')
    if j==0:
        ax.set(ylabel=r'[OIII] / $(\mathrm{H}\alpha + \mathrm{[NII]})$')
    ax.set_title(name)
    
    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    
    
h,l = fig.axes[0].get_legend_handles_labels()
fig.legend(h, l, bbox_to_anchor=(0., 1.01, 1., .051),loc = 'upper center',ncol=4)

plt.tight_layout()
filename = basedir / 'reports' / f'all_objects_line_diagnostics'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

## Combined Completeness limit

In [ ]:
path = basedir/'data'/'interim'
files = [x for x in path.iterdir() if x.stem.endswith('mock_sources')]

limit   = 0.8
max_sep = 0.3

ncols = 5
nrows = int(np.ceil(len(files)/ncols))

width = 2*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for file in files:
    
    ax = next(axes_iter)
    i, j = np.where(axes == ax)
    i,j=i[0],j[0]
    
    name = file.stem.split('_')[0]
    mock_sources = ascii.read(file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
  
    hist = []
    width = 0.5
    bins = np.arange(26,30,width)

    for center in bins:
        tmp = mock_sources[(mock_sources['magnitude']>center-width/2) & (mock_sources['magnitude']<=center+width/2)]
        if len(tmp)>0:
            hist.append(np.sum(tmp['sep']<max_sep)/len(tmp))
        else:
            hist.append(0)
    hist = np.array(hist)
    
    completeness_limit = np.max(bins[hist>=limit])
    
    ax.axhline(100*limit,color='black',lw=0.6)

    ax.bar(bins[hist>=limit],hist[hist>=limit]*100,width=width*0.9,color=tab10[0])
    ax.bar(bins[hist<limit],hist[hist<limit]*100,width=width*0.9,fc='white',ec=tab10[0])
    
    for b,h in zip(bins,hist):
        if b>=26.5:
            if h>=0.8:
                ax.text(b,5,f'{h*100:.0f}\%',horizontalalignment='center',color='white',fontsize=6)
            else:
                ax.text(b,5,f'{h*100:.0f}\%',horizontalalignment='center',color=tab10[0],fontsize=6)
    t = ax.text(0.75,0.92,f'{name}', transform=ax.transAxes,color='black',fontsize=7)
    t = ax.text(0.75,0.82,f'cl={completeness_limit:.0f}', transform=ax.transAxes,color='black',fontsize=7)

    ax.set(xlim=[26.2,29.8],
           ylim=[0,100])
    
    if i==nrows-1:
        ax.set(xlabel='m$_{[\mathrm{OIII}]}$')
    else:
        ax.set_xticklabels([])
    if j==0:
        ax.set(ylabel='percentage of recovered objects')
    else:
        ax.set_yticklabels([])
    
    #ax.set_title(f'{name}: cl = {completeness_limit}')
    
for i in range(nrows*ncols-len(files)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=0.05, hspace=0.05)

filename = basedir / 'reports' / f'all_galaxies_completeness'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

In [ ]:
from pnlf.detection import plot_completeness_limit

name = 'NGC0628'

file = basedir/'data'/'interim'/f'{name}_mock_sources.txt'

limit   = 0.8
max_sep = 0.3

mock_sources = ascii.read(file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')

filename = basedir / 'reports' / name / f'{name}_completness.pdf'

plot_completeness_limit(mock_sources,max_sep=max_sep,limit=limit,filename=filename)

## Plot RGB image

In [ ]:
from pnlf.auxiliary import circular_mask
from pnlf.plot.plot import create_RGB
from pnlf.io import ReadLineMaps

nrows = 5
ncols = 4
width = two_column
fig, axes_arr = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes = iter(axes_arr.flatten())

# loop over the galaxies we want to plot
for name in results['name']:  
        
    # get the next axis
    ax = next(axes)
    # find current position on the grid
    i, j = np.where(axes_arr == ax)
    i,j=i[0],j[0]
    
    #galaxy = galaxies[name]
    galaxy = ReadLineMaps(data_raw,name,**parameters[name])
    
    # define masks as slices
    masks = {
     'NGC1300' : circular_mask(*galaxy.shape,radius=50),
     'NGC1365' : circular_mask(*galaxy.shape,(720,420),radius=200),
     #'NGC1433' : circular_mask(*galaxy.shape,radius=100),
     'NGC1512' : circular_mask(*galaxy.shape,radius=70),
     'NGC1566' : circular_mask(*galaxy.shape,(450,450),radius=100)|circular_mask(*galaxy.shape,(350,150),radius=180),
     'NGC1672' : circular_mask(*galaxy.shape,(600,310),radius=60),
     #'NGC3627' : circular_mask(*galaxy.shape,(330,740),radius=100),
     'NGC3351' : circular_mask(*galaxy.shape,radius=200),
     'NGC4321' : circular_mask(*galaxy.shape,(550,450),radius=60),
     'NGC4535' : circular_mask(*galaxy.shape,(300,520),radius=100)
    }
    
    mask = np.zeros(galaxy.shape,dtype=bool)
    mask |= galaxy.star_mask.astype(bool)
    mask[masks.get(galaxy.name,(slice(-1,0),slice(-1,0)))] = True

    #img = galaxy.OIII5006_DAP.copy()
    img = create_RGB(galaxy.HA6562,galaxy.OIII5006_DAP,galaxy.SII6716,weights=[0.6,1,0.6],percentile=[95,99.,95])
    img[mask,...] = (1,1,1) #(1, 165/255, 1/255) 

    ax.imshow(img,origin='lower')
    ax.set_title(name)
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
filename = basedir / 'reports' / f'all_objects_rgb'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
#plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

## Overluminous sources

### RGB image

In [ ]:
exclude = {
 'IC5332'  : [1665,1335,618,538,615,876,665],
 'NGC0628' : [882,307,527],
 'NGC1300' : [1815,2704,2720],
 'NGC1365' : [732,743],
 'NGC1433' : [5657,4842], # both not at bright end
 'NGC1566' : [26,392,398],
 'NGC1672' : [138,198,273,154,257,136,169,162],
 'NGC2835' : [178,788,276,416,723],
 'NGC4254' : [1945,1944,2127,922],
 'NGC4303' : [381,412,457,430],
 'NGC5068' : [500,698,318,651,972,74,508,248,804,751],
 'NGC7496' : [131,464,569,126,550,438]
}

overluminous = {
 'NGC1512' : [277,272],
 'NGC1566' : [119,565,222],
 'NGC1672' : [203,211,124],
 'NGC2835' : [673],
 'NGC4303' : [371],
 'NGC4321' : [2111],
 'NGC7496' : [318,575],
}


In [ ]:
from pnlf.plot.plot import create_RGB
from pnlf.io import ReadLineMaps
from pnlf.auxiliary import filter_table
from pnlf.plot.plot import radial_profile


filename = basedir / 'reports' / f'all_galaxies_exclude'
objects = exclude

size = 40

n_objects = sum([len(v) for k,v in objects.items()])
print(f'plotting cutouts for {n_objects} objects')
ncols = 4
nrows = int(np.ceil(n_objects/ncols))

width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

failed = 0
for name, region_IDs in objects.items():
    
    galaxy = ReadLineMaps(data_raw/'MUSE_DR2'/'MUSEDAP',name,**parameters[name])
    
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
    
    for region_ID in region_IDs:
            
        try:
            row = filter_table(catalogue,id=region_ID)[0]
        except:
            print(f'error for {name}: {region_ID}')
            failed+=1
            continue
            
        ax = next(axes_iter)
        
        x,y = row[['x','y']]
        aperture_size=2.5*row['fwhm']/2
        
        star = Cutout2D(galaxy.OIII5006, (x,y), u.Quantity((size, size), u.pixel),wcs=galaxy.wcs)

        rgb = create_RGB(galaxy.HA6562,galaxy.OIII5006,galaxy.SII6716,percentile=99)
        yslice = slice(int(x-size/2),int(x+size/2))
        xslice = slice(int(y-size/2),int(y+size/2))
        
        ax.set_yticks([])
        ax.set_xticks([])
        
        try:
            im = ax.imshow(rgb[xslice,yslice,:],origin='lower')
        except:
            text = f'{name}: {row["id"]}'
            t = ax.text(0.06,0.87,text, transform=ax.transAxes,color='black',fontsize=7)
            continue
            
        aperture = CircularAperture((size/2+(x-int(x)),size/2+(y-int(y))),aperture_size)
        aperture.plot(color='tab:red',lw=0.8,axes=ax)
        
        profile = radial_profile(star.data,star.input_position_cutout)
        
        ax2 = ax.inset_axes([0.02, 0.02, 0.32, 0.25])
        ax2.set_yticks([])
        ax2.set_xticks([])  
        
        ax2.plot(profile,color='black')
        ax2.axvline(aperture_size,color='tab:red',lw=0.5)
  
        text = f'{name}: ID={row["id"]}'
        t = ax.text(0.07,0.87,text, transform=ax.transAxes,color='black',fontsize=7)
        t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
        
            
        if row['mOIII']<results.loc[name]['(m-M)']-3.97:
            for loc in ['bottom','top','right','left']:
                ax.spines[loc].set_color('tab:orange')
                ax.spines[loc].set_linewidth(1)
        #t = ax.text(0.05,0.8,f'mOIII={row["mOIII"]:.1f}', transform=ax.transAxes,color='black',fontsize=8)
        #t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
        
for i in range(nrows*ncols-n_objects+failed):
    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()
    
plt.subplots_adjust(wspace=-0.01,hspace=0.05)
if filename:
    #plt.savefig(filename.with_suffix('.png'),dpi=600)
    plt.savefig(filename.with_suffix('.pdf'),dpi=600)
plt.show()

### Spectrum of those objects
I only downloaded datacubes for the DR1 galaxies (IC5332,NGC0628,NGC1087,NGC1365,NGC1512,NGC1566,NGC1672,NGC2835,NGC3351,NGC3627,NGC4254,NGC4535,NGC5068)

In [ ]:
from pnlf.auxiliary import annulus_mask, circular_mask
from pnlf.io import ReadLineMaps
from pnlf.auxiliary import filter_table

name = 'NGC0628'

filename = basedir / 'reports' / name / f'{name}_exclude'
region_IDs = exclude[name]
xlim=[4750,7000]
size = 40

print(f'plot spectra for {len(region_IDs)} regions')

cube_path = Path('g:\Archive')/'MUSE'/'DR1'/'datacubes'
if name not in [x.stem.split('_')[0] for x in cube_path.iterdir()]:
    raise FileNotFoundError(f'no datacube for {name}')
with fits.open(cube_path / f'{name}_DATACUBE_FINAL.fits' , memmap=True, mode='denywrite') as hdul:
    data_cube   = hdul[1].data
    cube_header = hdul[1].header
    
galaxy = ReadLineMaps(data_raw/'MUSE_DR2'/'MUSEDAP',name,**parameters[name])

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)

fig = plt.figure(figsize=(two_column,len(region_IDs)*two_column/4)) 
gs = mpl.gridspec.GridSpec(len(region_IDs), 2, width_ratios=[1,3]) 

spectra = {}
for i,region_ID in enumerate(region_IDs):
    
    x,y = filter_table(catalogue,id=region_ID)[['x','y']][0]

    ax1 = fig.add_subplot(gs[2*i])
    ax2 = fig.add_subplot(gs[2*i+1])
    
    r = Cutout2D(galaxy.OIII5006, (x,y), u.Quantity((size, size), u.pixel),wcs=galaxy.wcs)
    norm = simple_norm(r.data,'linear',clip=False,percent=95)
    ax1.imshow(r.data, origin='lower',norm=norm,cmap='Greys')

    aperture = CircularAperture(r.position_cutout,8)
    aperture.plot(color='tab:red',lw=1,axes=ax1)

    t = ax1.text(0.07,0.87,f'{region_ID}', transform=ax1.transAxes,color='black',fontsize=7)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # there will be NaNs in the subcube that is used for the sigma clipping
        # astropy will issue a warning which we ignore in this enviornment
        circle  = circular_mask(*data_cube.shape[1:],(x,y),4)
        annulus = annulus_mask(*data_cube.shape[1:],(x,y),8,12) 
        _, bkg, _ = sigma_clipped_stats(data_cube[...,annulus],axis=1)
    
    spectrum = np.sum(data_cube[...,circle],axis=1)    
    # the background is the median * the number of non zero pixel
    spectrum_without_bkg = spectrum - bkg * np.sum(circle)
    
    #spectra = np.sum(data_cube[...,int(x)-1:int(x)+1,int(y)-1:int(y)+1],axis=(1,2))    
    # the wavelenght coverage of MUSE
    wavelength = np.linspace(4749.88,9349.88,data_cube.shape[0]) 
    
    ax2.plot(wavelength,spectrum,color=tab10[1],label='with background')
    ax2.plot(wavelength,spectrum_without_bkg,color=tab10[0],label='background subtracted')
    #ax2.legend() 

    ax2.set(xlim=xlim,
            ylabel=r'erg\,/\,s\,/\,\AA')
    if i ==0 :
        ax2.set_title(name)
    if i == len(region_IDs)-1:
        ax2.set(xlabel=r'$\lambda$\,/\,\AA')
    else:
        ax2.set_xticklabels([])
        
    ax2.yaxis.tick_right()
    ax2.yaxis.set_ticks_position('both')
    ax2.yaxis.set_label_position("right")
    
    # save spectra
    spectra[f'{region_ID}_wavelength'] = wavelength
    spectra[f'{region_ID}_spectra'] = spectrum
    spectra[f'{region_ID}_bkg'] = bkg * np.sum(circle)
    
plt.subplots_adjust(wspace=0,hspace=0.05)
    
if filename:
    plt.savefig(filename.with_suffix('.pdf'),dpi=600)
    
plt.show()

In [ ]:
from pnlf.auxiliary import annulus_mask, circular_mask

name = 'NGC0628'

cube_path = Path('g:\Archive')/'MUSE'/'DR1'/'datacubes'
if name not in [x.stem.split('_')[0] for x in cube_path.iterdir()]:
    raise FileNotFoundError(f'no datacube for {name}')
with fits.open(cube_path / f'{name}_DATACUBE_FINAL.fits' , memmap=True, mode='denywrite') as hdul:
    data_cube   = hdul[1].data
    cube_header = hdul[1].header
    
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    

def extract_spectra(cube,header,positions,region_ID,filename):
    '''extract spectra from a spectral cube at given positions'''
    
    logger.info(f'extracting spectrum for {len(positions)} objects')
    
    wavelength = []
    spectrum   = []
    background = []
    
    radii = [22.48, 21.42, 22.12]
    
    for i,pos in enumerate(positions):
        print(f'{i+1} of {len(positions)}')
        print(radii[i])
        x,y=pos
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # there will be NaNs in the subcube that is used for the sigma clipping
            # astropy will issue a warning which we ignore in this enviornment
            circle  = circular_mask(*cube.shape[1:],(x,y),radii[i])
            annulus = annulus_mask(*cube.shape[1:],(x,y),23,24) 
            _, bkg, _ = sigma_clipped_stats(cube[...,annulus],axis=1)

        spectrum.append(np.sum(data_cube[...,circle],axis=1))  
        background.append(bkg * np.sum(circle))
        wavelength.append(np.linspace(header['CRVAL3'],header['CRVAL3']+header['NAXIS3']*header['CD3_3'],header['NAXIS3']))
    
    spectra = Table(data=[region_ID,wavelength,spectrum,background],
                    names=['region_ID','wavelenght','spectrum','bkg'])

    hdu = fits.BinTableHDU(spectra,name='spectra')
    hdu.writeto(filename,overwrite=True)

In [ ]:
#positions = catalogue[catalogue['type']=='PN'][['x','y']]
#filename = basedir/'data'/'suspicious'/f'{name}_spectra_liz.fits'
#extract_spectra(data_cube,cube_header,positions[:50],catalogue[catalogue['type']=='PN']['id'][:50],filename)
    

In [ ]:
with fits.open(basedir/'data'/'suspicious'/'{}_spectra.fits'.format(name)) as hdul:
    spec = Table(hdul[1].data)
spec.add_index('region_ID')

In [ ]:
dap_path = Path('g:\Archive')/'MUSE'/'DR1'/'MUSEDAP'/'NGC0628_MAPS.fits'
with fits.open(dap_path) as hdul:
    wcs = WCS(hdul['FLUX'].header)
    

ra  = np.array([24.1623,24.1645,24.1888])
dec = np.array([15.7701,15.7958,15.7968])

p_sk = SkyCoord(ra*u.degree,dec*u.degree)

for sk in p_sk:
    p_xy.append(sk.to_pixel(wcs))

In [ ]:
filename = basedir/'data'/'suspicious'/f'{name}_spectra_liz.fits'
extract_spectra(data_cube,cube_header,positions,[1,2,3],filename)


In [ ]:
p_xy[0]

In [ ]:
x,y = p_xy[0]
width = 30
sub_cube = data_cube[:,int(x)-width:int(x)+width,int(y)-width:int(y)+width]

In [ ]:
with fits.open(basedir/'data'/'suspicious'/f'{name}_spectra_liz.fits') as hdul:
    spec = Table(hdul[1].data)
spec.add_index('region_ID')

In [ ]:
p = positions[0]

plt.plot(data_cube[:,int(p[0]),int(p[1])])

In [ ]:
sub = catalogue[(np.isin(catalogue['type'],['PN','SNR'])) & (catalogue['mOIII']<28)][['id','x','y','RaDec','mOIII','type']]

with open(basedir / 'data' / f'{name}_PN_and_SNR.txt','w',newline='\n') as f:
    ascii.write(sub,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')
    

In [ ]:
from spectral_cube import SpectralCube

In [ ]:
for sk in p_sk:
    sep=sk.separation(catalogue['SkyCoord'])
    row = catalogue[np.argmin(sep)]
    print(catalogue[sep.__lt__(Angle('10"'))]['type'])
    print(f'{np.min(sep.to(u.arcsec)):.2f}, {row["id"]}, {row["type"]}')

## Compare measured magnitudes

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)

from pnlf.load_references import NGC628, \
                                 pn_NGC628_kreckel, \
                                 snr_NGC628_kreckel, \
                                 pn_NGC628_herrmann, \
                                 NGC628_kreckel, \
                                 pn_NGC5068_herrmann, \
                                 pn_NGC3351_ciardullo, \
                                 pn_NGC3627_ciardullo

In [ ]:
fig,(ax1,ax2) = plt.subplots(nrows=2,ncols=1,figsize=(single_column,single_column*1.8))

'''
   NGC0628
'''

name = 'NGC0628'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[(catalogue['type']=='PN') | ((catalogue['type']=='SNR') & (catalogue['SNRorPN']==True)) ]
#catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]
catalogue['R2'] = catalogue['OIII5006'] / (catalogue['HA6562']+catalogue['NII6583'])
catalogue['dR2'] = catalogue['R2']  * np.sqrt(catalogue['OIII5006_err']/catalogue['OIII5006_err']**2 + 1/(catalogue['HA6562']+catalogue['NII6583'])**2 * (catalogue['HA6562_err']**2+catalogue['NII6583_err']**2) )                                  

matchcoord = NGC628.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
matchcoord['R_measured'] = catalogue[ID]['R2']
matchcoord['dR_measured'] = catalogue[ID]['dR2']
crit = sep.__lt__(Angle("0.5s"))

for s,c in zip(['Kreckel PN','Kreckel SNR','Herrmann PN'],['tab:red','tab:orange','tab:blue']):
    tmp = matchcoord[(matchcoord['source']==s) & crit]
    ax1.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
                 yerr = tmp['dmOIII_measured'],
                 marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label=s)
    tmp = matchcoord[(matchcoord['source']==s) & crit]
    ax2.errorbar(tmp['R'],tmp['R_measured'],
                 #xerr = tmp['dR'],
                 #yerr = tmp['dR_measured'],
                 marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label=s)


'''
   NGC5068
'''
name = 'NGC5068'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]
catalogue['R2'] = catalogue['OIII5006'] / (catalogue['HA6562']+catalogue['NII6583'])
catalogue['dR2'] = catalogue['R2']  * np.sqrt(catalogue['OIII5006_err']/catalogue['OIII5006_err']**2 + 1/(catalogue['HA6562']+catalogue['NII6583'])**2 * (catalogue['HA6562_err']**2+catalogue['NII6583_err']**2) )                                  

matchcoord = pn_NGC5068_herrmann.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
matchcoord['R_measured'] = catalogue[ID]['R2']
matchcoord['dR_measured'] = catalogue[ID]['dR2']
crit = sep.__lt__(Angle("0.5s"))

tmp = matchcoord[(crit)]
c = 'tab:green'
ax1.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
             yerr = tmp['dmOIII_measured'],
             #xerr = tmp['dmOIII'],
             marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label=s)
ax2.errorbar(tmp['R'],tmp['R_measured'],
             #xerr = tmp['sigma_R'],
             #yerr = tmp['dR_measured'],
             marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label=s)


'''
   NGC3627
'''
name = 'NGC3627'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]

matchcoord = pn_NGC3627_ciardullo.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
crit = sep.__lt__(Angle("0.5s"))

tmp = matchcoord[crit]
c = 'cyan'
ax1.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
             yerr = tmp['dmOIII_measured'],
             marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label='NGC3627')


'''
   NGC3351
'''
name = 'NGC3351'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]
catalogue['R2'] = catalogue['OIII5006'] / (catalogue['HA6562']+catalogue['NII6583'])
catalogue['dR2'] = catalogue['R2']  * np.sqrt(catalogue['OIII5006_err']/catalogue['OIII5006_err']**2 + 1/(catalogue['HA6562']+catalogue['NII6583'])**2 * (catalogue['HA6562_err']**2+catalogue['NII6583_err']**2) )                                  

matchcoord = pn_NGC3351_ciardullo.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
crit = sep.__lt__(Angle("0.5s"))

tmp = matchcoord[crit]
c = 'purple'
ax1.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
             yerr = tmp['dmOIII_measured'],
             marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label='NGC3351')


ax1.plot([25,27.5],[25,27.5],color='black',lw=0.4)
ax1.set(xlim=[25,27.5],ylim=[25,27.5])
ax1.set_xlabel(r'$\mathrm{m}_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ existing studies')
ax1.set_ylabel(r'$\mathrm{m}_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ this work')
ax1.legend()


xmin,xmax = 0,7
ymin,ymax = 0,7
ax2.plot([xmin,xmax],[xmin,xmax],color='black',lw=0.4)
ax2.set_xlim([xmin,xmax])
ax2.set_ylim([ymin,ymax])
ax2.set_xlabel(r'$I_{[\mathrm{O}\,\tiny{\textsc{iii}}]}\;/\;(I_{\mathrm{H}\,\alpha}+I_{[\mathrm{N}\,\tiny{\textsc{ii}}]})$ existing studies')
ax2.set_ylabel(r'$I_{[\mathrm{O}\,\tiny{\textsc{iii}}]}\;/\;(I_{\mathrm{H}\,\alpha}+I_{[\mathrm{N}\,\tiny{\textsc{ii}}]})$ this work')
#ax2.legend(loc=2)

plt.savefig(basedir / 'reports' / f'flux_comparison.pdf',dpi=600)
plt.show()


In [ ]:
name = 'NGC0628'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]
catalogue['R2'] = catalogue['OIII5006'] / (catalogue['HA6562']+catalogue['NII6583'])
catalogue['dR2'] = catalogue['R2']  * np.sqrt(catalogue['OIII5006_err']/catalogue['OIII5006_err']**2 + 1/(catalogue['HA6562']+catalogue['NII6583'])**2 * (catalogue['HA6562_err']**2+catalogue['NII6583_err']**2) )                                  

matchcoord = NGC628.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['IDf'] = ID
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
matchcoord['R_measured'] = catalogue[ID]['R2']
matchcoord['dR_measured'] = catalogue[ID]['dR2']
crit = sep.__lt__(Angle("0.5s"))


## measure (m-M) from inner and outer PNe

the metallicity and with it $M*$ decreases in the outer regions of the galaxies. A smaller $M*$ should lead to a larger distance. Therefore the PNe in the outer parts should yield a larger distance.

In [ ]:
abundance_gradients = ascii.read(basedir/'data'/'external'/'radial_abundance_gradients.txt',
                                names=['name','R0','g_r25'])
abundance_gradients.add_index('name')

pnlf_io = ascii.read(basedir/'data'/'interim'/ 'pnlf_io.txt')
pnlf_io.add_index('name')

In [ ]:
from regions import EllipseSkyRegion

name = 'NGC0628'

logOH_sun = 8.87
deltaM = lambda OH: 0.928*OH**2+0.225*OH+0.014

try:
    mu_trgb = trgb.loc[name]['trgb_(m-M)']
except:
    print(f'no TRGB for {name}')
    mu_trgb=0
completeness = parameters[name]['completeness_limit']
binsize = parameters[name]['binsize']

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    catalogue['overluminous'] = catalogue['overluminous'].astype(bool)

catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude']) & (~catalogue['overluminous']) & (catalogue['mOIII']<completeness)]

center = sample_table.loc[name]['SkyCoord']
posang = sample_table.loc[name]['posang']
inclination = sample_table.loc[name]['Inclination']
eccentricity = np.sin(inclination*u.deg).value

r25 = sample_table.loc[name]['r25']*u.arcmin

catalogue['r'] = catalogue['SkyCoord'].separation(center)
rmean = np.mean(catalogue['r']/r25).decompose()
logOH_rmean = abundance_gradients.loc[name]['R0']+rmean*abundance_gradients.loc[name]['g_r25']

print(f'rmean = {rmean:.2f} r25')
print(f'dM*={deltaM(logOH_rmean-logOH_sun):.3f}')

with fits.open(data_ext/'MUSE_DR2'/'MUSEDAP'/f'{name}_MAPS.fits') as hdul:
    wcs = WCS(hdul['OIII5006_FLUX '].header)
    OIII = hdul['OIII5006_FLUX'].data
    
catalogue['region'] = '     '

split into inner/outer

In [ ]:
threshold = 0.5
tried = set()
while True: 
    width = threshold*r25
    aperture = EllipseSkyRegion(center,
                            width=width,
                            height=np.sqrt((width)**2 * (1-eccentricity**2)),
                            angle=(posang-90)*u.deg)
    inside = aperture.contains(catalogue['SkyCoord'],wcs)
    ratio = np.sum(inside)/np.sum(~inside)
    
    if threshold in tried:
        break
    tried.add(threshold)

    if np.abs(ratio-1)<0.02:
        break
    elif ratio>1:
        threshold/=1.02
    elif ratio<1:
        threshold*=1.02

width = threshold*r25
aperture = EllipseSkyRegion(center,
                        width=width,
                        height=np.sqrt((width)**2 * (1-eccentricity**2)),
                        angle=(posang-90)*u.deg)
catalogue['region'] = 'inner'
catalogue['region'][~aperture.contains(catalogue['SkyCoord'],wcs)] = 'outer'
    
print(f'width={threshold:.3f} r25')

r = width/r25

logOH1 = abundance_gradients.loc[name]['R0']
logOH2 = abundance_gradients.loc[name]['R0']+r*abundance_gradients.loc[name]['g_r25']

print(f'inner dM*={deltaM(logOH1-logOH_sun):.3f}\noutter dM*={deltaM(logOH2-logOH_sun):.3f}')

or split into quadrants

In [ ]:
catalogue['region'] = 'west'

catalogue['region'][(center.ra<catalogue['SkyCoord'].ra)] = 'west'
catalogue['region'][(center.ra>catalogue['SkyCoord'].ra)] = 'east'


In [ ]:
catalogue['region'] = 'south'

catalogue['region'][(center.dec<catalogue['SkyCoord'].dec)] = 'north'
catalogue['region'][(center.dec>catalogue['SkyCoord'].dec)] = 'south'

In [ ]:
catalogue['region'] = 'south'
x0,y0 = center.to_pixel(wcs)
x,y = catalogue['x'],catalogue['y']

north = (y>y0-np.sin((posang)/180*np.pi)*(x-x0))
south = (y<y0-np.sin((posang)/180*np.pi)*(x-x0))
east = (x<x0+np.cos((posang-90)/180*np.pi)*(y-y0))
west = (x>x0+np.cos((posang-90)/180*np.pi)*(y-y0))

catalogue['region'][north & west] = 'nw'
catalogue['region'][north & east] = 'ne'
catalogue['region'][south & west] = 'sw'
catalogue['region'][south & east] = 'se'


compare central PNe to outer PNe

In [ ]:
x0,y0 = center.to_pixel(wcs)
dx = 1000

majx1 = x0-np.cos(posang/180*np.pi)*dx
majx2 = x0+np.cos(posang/180*np.pi)*dx
majy1 = y0-np.sin(posang/180*np.pi)*dx
majy2 = y0+np.sin(posang/180*np.pi)*dx

minx1 = x0-np.cos((posang-90)/180*np.pi)*dx
minx2 = x0+np.cos((posang-90)/180*np.pi)*dx
miny1 = y0-np.sin((posang-90)/180*np.pi)*dx
miny2 = y0+np.sin((posang-90)/180*np.pi)*dx


fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=wcs)

norm = simple_norm(OIII,clip=False,percent=97)
ax.imshow(OIII,norm=norm,cmap=plt.cm.Greys)

for region in np.unique(catalogue['region']):
    tmp = catalogue[catalogue['region']==region]
    ax.errorbar(tmp['x'],tmp['y'],fmt='o',ms=5,label=region)
pixel_aperture = aperture.to_pixel(wcs)
artist = pixel_aperture.as_artist(ec='black')
ax.add_artist(artist)

ax.plot([majx1,majx2],[majy1,majy2],color='black')
ax.plot([minx1,minx2],[miny1,miny2],color='black')

ax.set(xlim=(0,OIII.shape[1]),ylim=(0,OIII.shape[0]))
ax.set_title(name)
ax.legend()
plt.show()

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, pnlf, cdf
from pnlf.plot.pnlf import plot_pnlf
from scipy.stats import kstest

regions = np.unique(catalogue['region'])
if 'axes' in locals():
    del axes
for i,region in enumerate(regions):
    
    data = catalogue[catalogue['region']==region]['mOIII']
    err  = catalogue[catalogue['region']==region]['dmOIII']
    
    if len(data)<15:
        print(f'not enough data points ({len(data)}) for region {region}')
        continue
        
    fitter = MaximumLikelihood1D(pnlf,
                                 data[data<completeness],
                                 err=err[data<completeness],
                                 mhigh=completeness,Mmax=-4.47)
    mu,mu_p,mu_m = fitter([29])
    print('{}: {:.2f} + {:.2f} - {:.2f}'.format(region,mu,mu_p,mu_m))


    #Plot PNLF
    if 'axes' not in locals():
        axes = plot_pnlf(data,mu,completeness,
                 binsize=binsize,mhigh=28.5,Mmax=-4.47,color=tab10[i])
    else:
        axes = plot_pnlf(data,mu,completeness,
                         binsize=binsize,mhigh=28.5,Mmax=-4.47,filename=None,color=tab10[i],axes=axes)
    
ax1,ax2 = axes 
h, l = ax2.get_legend_handles_labels()
ax2.legend(h,regions)
filename=basedir/'reports'/name/f'{name}_PNLF_radial'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')

In [ ]:
row = [name,threshold,mui,muo,deltaM(logOH1-logOH_sun),deltaM(logOH2-logOH_sun)]
if name in pnlf_io['name']:
    pnlf_io.loc[name] = row 
else:
    pnlf_io.add_row(row)

In [ ]:
for col in pnlf_io.columns[1:]:
    pnlf_io[col].info.format='%.3f'
    
with open(basedir/'data'/'interim'/ 'pnlf_io.txt','w',newline='\n') as f:
    ascii.write(pnlf_io,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')

In [ ]:
pnlf_io['err(m-M)'] = 0.0

for row in pnlf_io:
    row['err(m-M)'] = np.sqrt(2)*results.loc[row['name']]['err+(m-M)']

In [ ]:
x = np.arange(len(pnlf_io))

pnlf_io.sort('r')
fig,ax=plt.subplots()

ax.axhline(0,color='black',lw=1)
ax.errorbar(x,pnlf_io['(m-M)outer']-pnlf_io['(m-M)inner'],
            yerr=pnlf_io['err(m-M)'],fmt='o',color='tab:red',label='(m-M)')
ax.scatter(x,pnlf_io['dM*inner']-pnlf_io['dM*outer'],color='tab:blue',label='$\Delta M*$')
ax.set_xticks(x)
ax.set_xticklabels(pnlf_io['name'],rotation=90)
ax.set(ylabel=r'outer - inner')
ax.legend()
plt.show()

In [ ]:
pnlf_io['d(m-M)'] = pnlf_io['(m-M)outer']-pnlf_io['(m-M)inner']
pnlf_io['dM*'] = pnlf_io['dM*outer']-pnlf_io['dM*inner']

for col in pnlf_io.columns[1:]:
    pnlf_io[col].info.format='%.3f'

In [ ]:
pnlf_io.sort('name')
ascii.write(pnlf_io[['name','r','d(m-M)','dM*']], sys.stdout, Writer = ascii.Latex,
            latexdict = {'tabletype': 'table*'})


## Metallicity dependance of the zero point

the zeropoint $M*$ of the PNLF must be measured from galaxies with known distances. Ciardullo+2002 suggested a quadratic dependence on metallicity. Here we try to validate this assumption by comparing our measured distances to TRGB distances and infer $M*$. 

From **Ferrares+2000**

The OIII5007 magnitude of the PNLF, m*, uncorrected for foreground extinction, is listed in column (9) of Table 3. Because PNLF distance moduli are calculated by Ðtting the luminosity function with a standard template (e.g., Ciardullo et al. 1989b), only the Ðnal distance moduli are published. From these we derived m* a posteriori by subtracting the zero point (and the extinction correction, if applied) adopted by the authors. The PNLF distances to the SMC, NGC 3109, and NGC 5253, listed in Table 3, are not well constrained. The planetary nebula (PN) sample in NGC 3109 (Richer & McCall 1992) includes only seven objects, and an upper limit to the distance is derived from the brightest of the PNs observed. Jacoby, Walker, & Ciardullo (1990) advise against the use of the PNLF distance to the SMC because of the small number of PNs deÐning the luminosity function. Finally, the small number of PNs detected in NGC 5253, the presence of strong internal dust extinction, and the galaxyÏs very low metal abundance all conjoin to produce a very ill constrained PNLF magnitude cuto†, unsuitable for distance determinations (Phillips et al. 1992). Uncertainties in the values of m* are summarized, for example, in Jacoby, Ciardullo, & Ford (1990). They include a contribution associated with the Ðtting procedure (of the order of 0.10 mag), photometric zero points (D0.05 mag), the Ðlter response calibration (D0.04 mag), and the uncertain deÐnition of the empirical PNLF (D0.05 mag). Errors in the reddening estimate, which are sometimes included, have been removed (in quadrature) from the present analysis, since we only deal with uncorrected magnitudes.

In [ ]:
# from Kreckel+2019
with open(basedir/'data'/'external'/'kreckel2019.txt') as f:
    txt=f.read()
txt = txt.replace('+or-','')
rows = txt.split('\n')
rows = [row.split('\t')[:-1] for row in rows]
        
new = []
for r1,r2 in zip(rows[::2],rows[1::2]):
    row = []
    for c1,c2 in zip(r1,r2):
        row.append(c1)
        if c2:
            row.append(c2)
    new.append(row)

names = ['name','R0_scal','R0_err_scal','g_r25_scal','g_r25_err_scal','g_kpc_scal','g_kpc_err_scal','sOH_scal',
                'R0_N2S2','R0_err_N2S2','g_r25_N2S2','g_r25_err_N2S2','g_kpc_N2S2','g_kpc_err_N2S2','sOH_N2S2',
                'R0_O3N2','R0_err_O3N2','g_r25_O3N2','g_r25_err_O3N2','g_kpc_O3N2','g_kpc_err_O3N2','sOH_O3N2',
                'R0_N2','R0_err_N2','g_r25_N2','g_r25_err_N2','g_kpc_N2','g_kpc_err_N2','sOH_N2']
met = Table( list(zip(*new)),names=names)
for c in met.columns[1:]:
    met[c] = met[c].astype(float)
met.add_index('name')

abundance_gradients = ascii.read(basedir/'data'/'external'/'radial_abundance_gradients.txt',
                                names=['name','R0','g_r25'])
abundance_gradients.add_index('name')

from astropy.table import join

trgb_distances = {'name':[],'trgb_(m-M)':[],'trgb_err(m-M)':[]}
for name in results['name']:
    litdist = distances = ascii.read(basedir / 'data' / 'literature distances' / f'{name}.csv',delimiter=',',comment='#')
    
    if 'TRGB' in litdist['Method']:
        sub = litdist[litdist['Method']=='TRGB']
        sub.sort('Refcode',reverse=True)
        
        trgb_distances['name'].append(name)
        trgb_distances['trgb_(m-M)'].append(sub['(m-M)'][0])
        trgb_distances['trgb_err(m-M)'].append(sub['err(m-M)'][0])
trgb = Table(trgb_distances)      
trgb = join(results[['name','(m-M)','err+(m-M)','err-(m-M)']],trgb,'name')
trgb = join(trgb,abundance_gradients,'name')
trgb.add_index('name')

trgb['rmean'] = 0.0
for name in trgb['name']:
    tmp = catalogue[(catalogue['Galaxy']==name) & (catalogue['notes']!='OL')]
    center = sample_table.loc[name]['SkyCoord']
    r25 = sample_table.loc[name]['r25']*u.arcmin
    trgb.loc[name]['rmean'] = np.mean(tmp['SkyCoord'].separation(center)/r25).decompose()
trgb['logOH'] = trgb['R0']+trgb['rmean']*trgb['g_r25']
trgb['dM*'] = trgb['(m-M)']-trgb['trgb_(m-M)']
trgb['errM*'] = np.sqrt(trgb['err+(m-M)']**2 + trgb['trgb_err(m-M)'])
trgb['errM*'].info.format = '%.2f'

In [ ]:
prescription = 'O3N2'
r = 0.5 # position in r25

deltaM = lambda OH: 0.928*OH**2+0.225*OH+0.014

logOH_sun = 8.87
logOH = np.linspace(7.8,9.5)

fig,((ax1,ax2),(ax3,ax4)) = plt.subplots(nrows=2,ncols=2,figsize=(two_column,two_column/1.618))

for ax,prescription in zip([ax1,ax2,ax3,ax4],['scal','N2S2','O3N2','N2']):
    ax.plot(logOH,-4.47+deltaM(logOH-logOH_sun),color='black',ls=':')

    for n in list(met['name']):
        row = met.loc[n]
        x = row[f'R0_{prescription}']+r*row[f'g_r25_{prescription}']
        y = -4.47+deltaM(x-logOH_sun)
        xerr = np.sqrt(row[f'R0_err_{prescription}']**2+(r*row[f'g_r25_err_{prescription}'])**2)
        yerr = np.sqrt((2*0.928*x+0.225)**2 *xerr**2)

        ax.errorbar(x,y,fmt='o',ms=4,
                    xerr=xerr,yerr=yerr,label=n)    
    ax.axhline(-4.47,ls='--',lw=0.5)
    ax.set(xlim=[8,9],ylim=[-4.6,-4.1],xlabel=f'12+log O/H (at {r} r25)',ylabel='M* (mag)')
    ax.invert_yaxis()
    ax.set_title(prescription)
    
plt.legend(ncol=3)
plt.tight_layout()
#plt.savefig(basedir/'reports'/'zero_point.pdf',dpi=600)
plt.show()

### fix $(m-M)$ and fit $M*$

we fix $(m-M)$ to the TRGB value and leave $M*$ as a free parameter 

IC5332,NGC0628,NGC1365,NGC1433,NGC1512,NGC1566,NGC2835,NGC3351,NGC3627,NGC4321,NGC5068

In [ ]:
name = 'NGC3351'

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    catalogue['overluminous'] = catalogue['overluminous'].astype(bool)

catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude']) & (~catalogue['overluminous']) & (catalogue['mOIII']<completeness)]


In [ ]:
from pnlf.plot.pnlf import plot_pnlf
from pnlf.analyse import F
from scipy.optimize import minimize

def pnlf_Mmax(m,Mmax,mu,mhigh):

    m = np.atleast_1d(m)
    mlow = Mmax+mu
    
    normalization = 1/(F(mhigh,mu) - F(mlow,mu))    
    out = normalization * np.exp(0.307*(m-mu)) * (1-np.exp(3*(Mmax-m+mu)))
    out[(m>mhigh) | (m<mlow)] = 0
    
    return out

def gaussian(x,mu,sig):
    return 1/np.sqrt(2*np.pi*sig**2) * np.exp(-(x-mu)**2/(2*sig**2))

def prior(param):
    return gaussian(param,-4.47,0.08)

# pre-process the data for the plot and read additional parameters
data = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude'])]['mOIII']
err = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude'])]['dmOIII']

mu_trgb = trgb.loc[name]['trgb_(m-M)']
completeness = parameters[name]['completeness_limit']
binsize = parameters[name]['binsize']

fitter = MaximumLikelihood1D(pnlf_Mmax,data[data<completeness],err=err[data<completeness],mu=mu_trgb,mhigh=completeness)
Mmax = minimize(fitter.likelihood,[-4.47],method=fitter.method).x[0]

mlow = Mmax+mu_trgb
mhigh = 28.5

print(f'{name}: Mmax={Mmax:.2f}, dMmax={Mmax+4.47:.2f}')

axes = plot_pnlf(data,mu_trgb,completeness,binsize=binsize,mhigh=28.5,Mmax=Mmax,filename=None,color=tab10[0])

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, F
from pnlf.plot.pnlf import _plot_pnlf
from scipy.optimize import minimize

names = list(trgb['name'])

names.remove('NGC1433')
names.remove('NGC1512')

nrows = 3
ncols = 3
filename = None #basedir / 'reports' / f'all_galaxies_PNLF'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------

width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

trgb['dM*new'] = 0.0
for name in names:
    
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
    else:
        print(f'no catalogue for {name}')
        continue
        
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0
        
    # pre-process the data for the plot and read additional parameters
    data = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude'])]['mOIII']
    err = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude'])]['dmOIII']

    mu_trgb = trgb.loc[name]['trgb_(m-M)']
    completeness = parameters[name]['completeness_limit']

    binsize = parameters[name]['binsize']

    fitter = MaximumLikelihood1D(pnlf_Mmax,data[data<completeness],err=err[data<completeness],prior=prior,mu=mu_trgb,mhigh=completeness)
    Mmax = minimize(fitter.likelihood,[-4.47],method=fitter.method).x[0]
    
    trgb.loc[name]['dM*new'] = Mmax+4.47
    
    mlow = Mmax+mu_trgb
    mhigh = 28.5
    
    print(f'{name}: Mmax={Mmax:.2f}, dMmax={Mmax+4.47:.2f}')
    
    ax=_plot_pnlf(data,mu_trgb,completeness,binsize=binsize,mlow=mlow,mhigh=mhigh,ax=ax,ms=3)
    ax.text(0.4,0.08,f'{name}', transform=ax.transAxes,fontsize=7)

    #ax.set_xlim([mu-5,completeness+0.5])
    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'N')
    #ax.set_title(name)
    #ax.set(xlim=[24,28.5])
    
plt.show()

### Fit dM* to log (O/H)

In [ ]:
from astropy.modeling import models, fitting

# OH = logOH-logOH_sun
deltaM = lambda OH: 0.928*OH**2+0.225*OH+0.014

logOH_sun = 8.87
logOH = np.linspace(8,9)

model  = models.Polynomial1D(degree=2,c0=0.014, c1=0.225, c2= 0.928)
fitter = fitting.LinearLSQFitter()

mask = np.isin(trgb['name'],['NGC1433','NGC1512'])

fit = fitter(model,trgb[~mask]['logOH']-logOH_sun,trgb[~mask]['dM*'],weights=trgb[~mask]['errM*'])

print('Ciardullo: c0=0.014, c1=0.225, c2= 0.928')
print('Fit:       c0={:.3f}, c1={:.3f}, c2={:.3f}'.format(*fit.parameters))

fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(two_column,two_column/1.6))

ax.errorbar(trgb[~mask]['logOH'],-4.47+trgb[~mask]['dM*'],yerr=trgb[~mask]['errM*'], fmt='ko',label='data')
ax.errorbar(trgb[mask]['logOH'],-4.47+trgb[mask]['dM*'],yerr=trgb[mask]['errM*'], fmt='ro',label='excluded')
ax.plot(logOH, -4.47+fit(logOH-logOH_sun), 'b-', lw=2,label='fit')
ax.plot(logOH,-4.47+deltaM(logOH-logOH_sun),'k:',label='Ciarduollo+2002')

ax.axhline(-4.47,ls='--',color='k')
#ax.set(xlabel='12+log O/H',ylabel='M* (mag)')
ax.set(xlim=[8,9],ylim=[-6.5,-2.5],xlabel=f'12+log O/H',ylabel='M* (mag)')
ax.invert_yaxis()
plt.legend()
plt.show()

In [ ]:
trgb

In [ ]:
cepheids = Table({
 'name':['LMC','SMC','NGC224','NGC300','NGC598','NGC2403','NGC3031','NGC3351','NGC3368','NGC3627','NGC4258','NGC5253','NGC5457'],
 'EBV' : [0.075,0.037,0.062,0.013,0.041,0.040,0.080,0.028,0.025,0.032,0.016,0.056,0.009],
 '(m-M)' : [18.50,19.01 ,24.38 ,26.53 ,24.56 ,27.48 ,27.75 ,29.85 ,29.97 ,29.86 ,29.44 ,27.56 ,29.13],
 'err(m-M)' : [0.0,0.03,0.05,0.07,0.10,0.10,0.08,0.09,0.06,0.08,0.07,0.14,0.11],
 'M*': [-4.56,-4.67,-4.66,-4.21,-4.08,-4.41,-4.52,-4.39,-4.65,-4.44,-4.51,-4.05,-4.28],
 '+M*': [0.13,0.40,0.14,0.67,0.16,0.16,0.12,0.19,0.12,0.12,0.13,0.63,0.15],
 '-M*' : [0.09,0.17,0.11,0.16,0.14,0.13,0.11,0.13,0.11,0.12,0.11,0.16,0.14],
 'logOH': [8.50,8.03,8.98,8.35,8.82,8.80,8.75,9.24,9.20,9.25,8.85,8.15,8.50],
 'dM*' : [0.06,0.48,0,0.15,0,0,0,0,0,0,0,0.33,0.06]
})

zeropoint = cepheids.copy()
# dM* has a different meaning in the Cepheid table (some correction form Dopita+92)
trgb['M*'] = -4.47+trgb['dM*']
# add our own data
for row in trgb:
    if row['name'] not in ['NGC1433','NGC1512']:
        new = [row['name'],0,row['(m-M)'],row['err+(m-M)'],row['M*'],row['errM*'],row['errM*'],row['logOH'],0]
        zeropoint.add_row(new)
    
err_p,err_m = zeropoint['+M*'], zeropoint['-M*']

zeropoint.sort('logOH')

In [ ]:
model  = models.Polynomial1D(degree=2,c0=0.014, c1=0.225, c2= 0.928)
fitter = fitting.LinearLSQFitter()

logOH = np.linspace(7.8,9.5)

fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(single_column,single_column/1.5))

fit = fitter(model,cepheids['logOH']-logOH_sun,cepheids['M*']+cepheids['dM*']+4.47,weights=1/cepheids['+M*'])
print('Ciardullo: c0=0.014, c1=0.225, c2= 0.928')
print('Fit Cepheids: c0={:.3f}, c1={:.3f}, c2={:.3f}'.format(*fit.parameters))
#ax.plot(logOH, -4.47+fit(logOH), 'b-', lw=2,label='fit Cepheids')

fit = fitter(model,zeropoint['logOH']-logOH_sun,zeropoint['M*']+zeropoint['dM*']+4.47,weights=1/zeropoint['+M*'])
print('Fit Cepheids+TRGB: c0={:.3f}, c1={:.3f}, c2={:.3f}'.format(*fit.parameters))

ax.errorbar(cepheids['logOH'],cepheids['M*']+cepheids['dM*'],yerr=[cepheids['+M*'],cepheids['-M*']], fmt='o',color=tab10[1],ms=3,label='Cepheids')
ax.errorbar(trgb[~mask]['logOH'],trgb[~mask]['M*'],yerr=trgb[~mask]['errM*'], fmt='o',color=tab10[0],ms=3,label='TRGB')

ax.plot(logOH, -4.47+fit(logOH-logOH_sun), 'k-', lw=2,label='fit')
ax.plot(logOH,-4.47+deltaM(logOH-logOH_sun),'k:',lw=2,label='Ciardullo+2002')
plt.locator_params(axis='y',nbins=5)

# and now with a constant line
model  = models.Polynomial1D(degree=1,c0=-4.5, c1=0)
model.c1.fixed=True
fit = fitter(model,trgb[~mask]['logOH'],trgb[~mask]['M*'],weights=1/trgb[~mask]['errM*'])
print(f'TRGB: M*={fit.c0.value:.2f}')

fit = fitter(model,cepheids['logOH'],cepheids['M*'],weights=1/cepheids['+M*'])
print(f'Cepheids: M*={fit.c0.value:.2f}')

fit = fitter(model,zeropoint['logOH'],zeropoint['M*'],weights=1/zeropoint['+M*'])
print(f'together: M*={fit.c0.value:.2f}')

#for row in cepheids:
#    ax.text(row['logOH'],row['M*']+row['dM*'],row['name'])
ax.axhline(fit.c0.value,ls='--',lw=1,color='k')

#ax.set(xlabel='12+log O/H',ylabel='M* (mag)')
ax.set(xlim=[8,9.4],ylim=[-5.5,-3.5],xlabel=f'12+log O/H',ylabel='M* (mag)')
ax.invert_yaxis()
plt.legend(ncol=2)
plt.savefig(basedir/'reports'/'zeropoint.pdf',dpi=600)
plt.show()

In [ ]:
catalogue = ascii.read(basedir/'data'/'catalogues'/'PN_candidates.txt')
catalogue['SkyCoord'] = SkyCoord(catalogue['R.A.'],catalogue['Dec.'])

## Redo all distance plots

In [ ]:
from pnlf.plot.pnlf import compare_distances

name = results[14]['name']
mu,mu_m,mu_p = results.loc[name][['(m-M)','err-(m-M)','err+(m-M)']]

print(name)
filename = basedir / 'reports' / name / f'{name}_distances'
distances = compare_distances(name,mu,mu_p,mu_m,filename=filename)

## PN Number vs Mass etc.

In [ ]:
sample=ascii.read(basedir/'reports'/'sample.txt')
sample['SkyCoord'] = SkyCoord(sample['R.A.'],sample['Dec.'])
sample.add_index('Name')

In [ ]:
galaxies = {}

for n in result['name']:
    print('are you sure that you want to run this?')
    break
    filename = data_raw / 'MUSEDAP' / f'{n}_MAPS.fits'

    with fits.open(filename) as hdul:
        d=hdul['STELLAR_MASS_DENSITY'].data
        galaxies[n]= np.nansum(d)

In [ ]:
result['mass']=sample['mass']
result['Survey Area'] = sample['Survey Area']

Calculate the survey area from number of pixels and distance

In [ ]:
for row in result:
    N_pix = row['N_pixel']
    d = Distance(distmod=parameters[row['name']]['mu'])
    A_pix = ((d*(0.2/206265))**2).to(u.kpc**2)
    
    print(f'{row["name"]}: {d:.2f}, {N_pix*A_pix:.2f}')

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,nrows=1,figsize=(12,4))

for row in result:
    if row['N_PN']>1:
        ax1.scatter(row['mass'],row['N_PN'])
        ax1.text(row['mass'],row['N_PN']+2,row['name'],horizontalalignment='center')
        
        
        ax2.scatter(row['Survey Area'],row['N_PN'])
        ax2.text(row['Survey Area'],row['N_PN']+2,row['name'],horizontalalignment='center') 

        ax3.scatter(row['N_pixel'],row['N_PN'])
        ax3.text(row['N_pixel'],row['N_PN']+2,row['name'],horizontalalignment='center') 
        

ax1.set(xlabel='stellar mass density',ylabel='N PN')
ax2.set(xlabel='Survey Area')
ax3.set(xlabel='Npixel')
#ax3.set(xlabel='Npixel')
plt.show()

## Other PNLF studies

In [ ]:
distances = ascii.read(basedir/'data'/'literature distances'/'latest.csv',delimiter=',',header_start=12,data_start=14)
results = ascii.read(basedir/'data'/'interim'/'results.txt')
print(f"intial cagalogue has {len(np.unique(distances[distances['Method']=='PNLF']['Galaxy ID']))} objects")

In [ ]:
pnlf_distances = distances[distances["Method"]=='PNLF']
print(f"excluding {len(np.unique(pnlf_distances[(pnlf_distances['m-M']<22.5) | (pnlf_distances['m-M']>32)]['Galaxy ID']))} objects")
pnlf_distances = pnlf_distances[(pnlf_distances['m-M']>22.5) & (pnlf_distances['m-M']<32)]  # exclude the many measurments of the LMC and SMC

pnlf_distances['year'] = pnlf_distances['Date (Yr. - 1980)']+1980
pnlf_distances.rename_column('Galaxy ID','name')
pnlf_distances['name'] = [n.rstrip('a').rstrip('b') for n in pnlf_distances['name']]

alias = {
    'NGC 0628': 'MESSIER 074',
    'NGC 3351': 'MESSIER 095',
    'NGC 3627': 'MESSIER 066',
    'NGC 4254': 'MESSIER 099',
    'NGC 4303': 'MESSIER 061',
    'NGC 4321': 'MESSIER 100'
}

alias_back = {v:k for k,v in alias.items()}

phangs_sample = []
for row in results:
    name = row['name'].replace('NGC','NGC ').replace('IC','IC ')
    name = alias.get(name,name)
    phangs_sample.append(name)
    new = ['',0,0,name,row['(m-M)'],row['err-(m-M)'],0,'PNLF','Schmnn+2020','',0,0,0,40,'',2020]
    pnlf_distances.add_row(new)
    
galaxies = list(np.unique(pnlf_distances['name']))

In [ ]:
# we sort by mean distance
mean_dis = []
for gal in galaxies:
    mean_dis.append(pnlf_distances[pnlf_distances['name']==gal]['m-M'].mean())

In [ ]:
pnlf_distances.sort('year')
important_papers = dict()
is_measured = []
for row in pnlf_distances:
    #if row['name'] not in is_measured:
    #    is_measured.append(row['name'])
    if row['REFCODE'] in important_papers:
        important_papers[row['REFCODE']] += 1
    else:
        important_papers[row['REFCODE']] = 1


In [ ]:

fig, ax = plt.subplots(1,figsize=(10,6/1.618))

x_pos  = []
gal_names = [x for _,x in sorted(zip(mean_dis,galaxies))]

print(f'{len(gal_names)} galaxies in sample')
color_grid=[]
for i,gal in enumerate(gal_names):
    tmp = pnlf_distances[pnlf_distances['name']==gal]
    ax.scatter(len(tmp)*[i+1],tmp['m-M'],marker="_",color='gray')
    # color the measured by me red
    if len(tmp[tmp['REFCODE']=='Schmnn+2020'])>0:
        ax.scatter([i+1],tmp[tmp['REFCODE']=='Schmnn+2020']['m-M'],marker="_",color='tab:red')
        color_grid.append(i)

# create a legend with numbers only
#legend_elements = [mpl.lines.Line2D([0], [0], color=tab10[i], lw=2, label=str(i+1)) for i in range(10)]
#plt.legend(handles=legend_elements, loc='upper center',ncol=10)
    
ymin,ymax = 23,32
# set the galaxy names as x-ticklabels
ax.set(xticks=np.arange(1,len(galaxies)+1),
       ylabel='($m-M$) / mag',
       title='Galaxies with PNLF distances',
       xlim=[0.5,len(galaxies)+0.5],
       ylim=[ymin,ymax])
ax.set_xticklabels(gal_names,rotation=90,color='gray')    

# color the galaxies which are in the phangs sample 
for n in phangs_sample:
    i = gal_names.index(n)
    ax.get_xticklabels()[i].set_color("tab:red")
#for n in ['MESSIER 066','MESSIER 074','MESSIER 095','NGC 5068']:
#    i = gal_names.index(n)
#    ax.get_xticklabels()[i].set_color("tab:red")    

ax.grid(axis='x')
#a = ax.get_xgridlines()
#for i in color_grid:
#    a[i].set_color('tab:red')

yticks_mpc = np.logspace(np.log10(Distance(distmod=ymin).to(u.Mpc).value),np.log10(Distance(distmod=ymax).to(u.Mpc).value),10)
yticks_mu  = Distance(yticks_mpc*u.Mpc).distmod
    
ax2 = ax.twinx()
ax2.set_yticks(yticks_mu.value,minor=False)
ax2.set_yticklabels([f'{x:.2f}' for x in yticks_mpc],ha="left")
ax2.set(ylim=[ymin,ymax],ylabel='$D$ / Mpc')

plt.tight_layout()

plt.savefig(basedir/'reports'/'PNstudies.png',dpi=600)
plt.show()

In [ ]:

fig, (ax2,ax1) = plt.subplots(nrows=2,figsize=(two_column,two_column))

gal_names = [x for _,x in sorted(zip(mean_dis,galaxies))]

def literature_distances(labels,ax,ymin=23,ymax=32):
    
    # set the galaxy names as x-ticklabels
    ax.set(xticks=np.arange(1,len(labels)+1),
           ylabel='($m-M$) / mag',
           xlim=[0.5,len(labels)+0.5],
           ylim=[ymin,ymax])
    labels_new = [alias_back.get(l,l) for l in labels]
    ax.set_xticklabels(labels_new,rotation=90,color='gray')    
    ax.grid(axis='x',ls='--',lw=0.4)
    grid = ax.get_xgridlines()

    for i,label in enumerate(labels):
        tmp = pnlf_distances[pnlf_distances['name']==label]
        ax.scatter(len(tmp)*[i+1],tmp['m-M'],marker="_",color='gray')
        # color the measured by me red
        if len(tmp[tmp['REFCODE']=='Schmnn+2020'])>0:
            ax.get_xticklabels()[i].set(color="tab:red",fontweight='black')
            ax.scatter([i+1],tmp[tmp['REFCODE']=='Schmnn+2020']['m-M'],marker="_",color='tab:red')
            grid[i].set(ls='-',lw=0.5)
    
    
    yticks_mpc = np.logspace(np.log10(Distance(distmod=ymin).to(u.Mpc).value),np.log10(Distance(distmod=ymax).to(u.Mpc).value),10)
    yticks_mu  = Distance(yticks_mpc*u.Mpc).distmod

    axt = ax.twinx()
    axt.set_yticks(yticks_mu.value,minor=False)
    axt.set_yticklabels([f'{x:.2f}' for x in yticks_mpc],ha="left")
    axt.set(ylim=[ymin,ymax],ylabel='$D$ / Mpc')
    
    return ax
    
n = len(gal_names)
ax1 = literature_distances(gal_names[:int(n/2)],ax1,ymin=23,ymax=30.5)
ax2 = literature_distances(gal_names[int(n/2):],ax2,ymin=29.5,ymax=32.2)

#ax2.set_title('Galaxies with PNLF distances')
plt.tight_layout()


plt.savefig(basedir/'reports'/'PNstudies.pdf',dpi=600)
plt.savefig(basedir/'reports'/'PNstudies.png',dpi=600)

plt.show()

In [ ]:
from scipy.stats import mode

In [ ]:
def mode_only(array):
    m,c = mode(array)
    return m

## Compare with Deep

In [ ]:
deep   = ascii.read(basedir/'data'/'literature distances'/'deep_distances.csv')
result = ascii.read(basedir/'data'/'interim'/'results.txt')

deep.add_index("galaxy")
deep['d'] = [float(x[:-4]) for x in deep['distance']]
deep['e'] = [float(x[:-4]) for x in deep['error']]

In [ ]:
from pnlf.auxiliary import mu_to_parsec

fig,ax = plt.subplots(figsize=(two_column,two_column/1.618))
result.sort('d/Mpc')
for i, row in enumerate(result):
    d,(dp,dm) = mu_to_parsec(row['(m-M)'],[row['err+(m-M)'],row['err-(m-M)']])

    ax.errorbar(i-0.1,row['d/Mpc'],yerr=([dm.value],[dp.value]),fmt='o',color='tab:red')
    
    tmp = deep.loc[row['name']]
    ax.errorbar(i+0.1,tmp['d'],yerr=tmp['e'],fmt='o',color='black')
    
ax.set(xticks=np.arange(0,len(result)),
       ylabel='$D$ / Mpc',
       title='PHANGS distances',
       xlim=[-0.5,len(result)-0.5])
ax.set_xticklabels(result['name'],rotation=90)  
ax.grid(axis='x')

legend_elements = [mpl.lines.Line2D([0], [0], color=col, lw=2, label=l) for col,l in zip(['tab:red','black'],['PNLF','TRGB'])]
plt.legend(handles=legend_elements, loc='lower center',ncol=10)
plt.savefig(basedir/'reports'/'PNLF_vs_TRGB.png',dpi=600)

plt.show()

## Plot entire sky

In [ ]:
filename = basedir / 'data' / 'interim' / 'sample.txt'
sample = ascii.read(filename,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
sample['SkyCoord'] = SkyCoord(sample['R.A.'],sample['Dec.'])

ra = sample['SkyCoord'].ra
ra = ra.wrap_at(180*u.degree)
dec = sample['SkyCoord'].dec

In [ ]:
#mpl.use('pdf')
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(ra.radian,dec.radian,marker='.')
#ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)
ax.set_xticklabels([])
ax.set_yticklabels([])

for x,y,s in zip(ra,dec,sample['Name']):
    ax.annotate(s,(x.radian,y.radian),xycoords='data',size='x-small')

fig.savefig("map.pdf")


In [ ]:
# https://github.com/henrysky/milkyway_plot/blob/master/mw_plot/mw_plot_classes.py
image_filename = basedir/'data'/'interim'/'MW_edgeon_unannotate.jpg'
img = plt.imread(image_filename)
img = img[1625:4875]  # so there are 3250px there

center=(0, 0) * u.deg
radius=(180, 90) * u.deg
    
y_img_center = 1625 - int((3250 / 180) * center[1].value)
y_radious_px = int((3250 / 180) * radius[1].value)
x_img_center = int((6500 / 360) * center[0].value) + 3250
x_radious_px = int((6500 / 360) * radius[0].value)

img = img[(y_img_center - y_radious_px):(y_img_center + y_radious_px),
             (x_img_center - x_radious_px):(x_img_center + x_radious_px), :]

'''
fig = plt.figure()
ax = fig.add_subplot(111, projection='mollweide')

lon = np.linspace(-np.pi, np.pi, 6500)
lat = np.linspace(np.pi / 2., -np.pi / 2., 3250)
Lon, Lat = np.meshgrid(lon, lat)

plt.show()
'''

In [ ]:
positions = {'IC5332':(4,2),
'NGC0628':(0,1),
'NGC1087':(0,2),
'NGC1300':(0,3),
'NGC1365':(4,3),
'NGC1385':(0,4),
'NGC1433':(4,5),
'NGC1512':(4,4),
'NGC1566':(3,6),
'NGC1672':(4,6),
'NGC2835':(2,6),
'NGC3351':(1,6),
'NGC3627':(0,6),
'NGC4254':(1,0),
'NGC4303':(3,0),
'NGC4321':(0, 0),
'NGC4535':(2,0),
'NGC5068':(4,0),
'NGC7496':(4,1)}

va = {
'IC5332':'bottom',
'NGC0628':'bottom',
'NGC1087':'bottom',
'NGC1300':'center',
'NGC1365':'center',
'NGC1385':'center',
'NGC1433':'center',
'NGC1512':'center',
'NGC1566':'center',
'NGC1672':'center',
'NGC2835':'center',
'NGC3351':'top',
'NGC3627':'bottom',
'NGC4254':'top',
'NGC4303':'top',
'NGC4321':'bottom',
'NGC4535':'center',
'NGC5068':'bottom',
'NGC7496':'top'}
ha = {
'IC5332':'center',
'NGC0628':'center',
'NGC1087':'center',
'NGC1300':'right',
'NGC1365':'right',
'NGC1385':'left',
'NGC1433':'right',
'NGC1512':'left',
'NGC1566':'right',
'NGC1672':'left',
'NGC2835':'right',
'NGC3351':'right',
'NGC3627':'right',
'NGC4254':'left',
'NGC4303':'left',
'NGC4321':'left',
'NGC4535':'left',
'NGC5068':'center',
'NGC7496':'center'}

In [ ]:
from pnlf.plot.plot import create_RGB


fig, axs = plt.subplots(ncols=7, nrows=5,figsize=(1.5*two_column,two_column))
gs = axs[1, 2].get_gridspec()

path = data_raw / 'MUSE_DR2' / 'filterImages' 

# remove the underlying axes
for ax in axs[1:-1,1:-1].flatten():
    ax.remove()
ax = fig.add_subplot(gs[1:-1,1:-1],projection="mollweide")
ax.pcolormesh(Lon, Lat,img[:, :, 0], cmap='gray', zorder=2, alpha=0.85, rasterized=True)
ax.plot(ra.radian,dec.radian,'.r',ms=1)

#ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)
ax.set_xticklabels([])
ax.set_yticklabels([])

for x,y,name in zip(ra,dec,sample['Name']):
    #ax.annotate(s,(x.radian,y.radian),xycoords='data',size='x-small',position='top')
    ax.text(x.radian,y.radian,name,
            horizontalalignment=ha[name],
            verticalalignment=va[name],
            fontsize=6,
            color='white')

for name,idx in positions.items():

    sdss_g, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_g_WCS_Pall_mad.fits',header=True)
    sdss_r, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_r_WCS_Pall_mad.fits',header=True)
    sdss_i, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_i_WCS_Pall_mad.fits',header=True)
    
    #ax=axs[idx]
    axs[idx].remove()
    ax = fig.add_subplot(gs[idx],projection=WCS(h))
    
    gri = create_RGB(sdss_i,sdss_r,sdss_g,weights=[1,1,1],percentile=[99,99,99])
    gri[sdss_g==0] = (1,1,1)
    ax.imshow(gri)
    
    #ax.annotate(f'{k}',(0.1, 0.5),xycoords='axes fraction', va='center')
    ax.set_title(name,fontsize=6)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_axis_off()
    
axs[(0,5)].remove()

#fig.tight_layout()
plt.subplots_adjust(wspace=0,hspace=0.3)

plt.savefig(basedir/'reports'/'all_galaxies_sky.pdf',dpi=600)
plt.show()

## Angular resolution of all galaxies

In [ ]:
for name in results['name']:
    with fits.open(data_raw/'AUXILIARY'/'seeing_maps'/f'{name}_seeing.fits') as hdul:
        PSF = hdul[0].data
    
    res_min = np.nanmin(PSF)/206265*results.loc[name]['d/Mpc']*1e6
    res_max = np.nanmax(PSF)/206265*results.loc[name]['d/Mpc']*1e6
    
    ang_min = np.nanmin(PSF)
    ang_max = np.nanmax(PSF)
    
    print(f'{name}: min={ang_min:.2f} pc, max={ang_max:.2f} pc')

## Misc

In [ ]:
from pnlf.auxiliary import mu_to_parsec, parsec_to_mu

In [ ]:
d = 18.72
mu,mu_err = parsec_to_mu(d*u.Mpc,0.15*d*u.Mpc)
print(f'{mu.value:.2f},{mu_err[0]:.2f},{d:.2f},NAM,2020AJ....159...67K,,,,,')

In [ ]:
d,derr= 17.22, 2.58 
mu,mu_err = parsec_to_mu( d*u.Mpc,derr*u.Mpc)
print(f'{mu.value:.2f},{mu_err[0]:.2f},{d:.2f}')

In [ ]:
trgb['(m-M)'], trgb['err(m-M)'] = parsec_to_mu(trgb['Distance']*u.Mpc,trgb['Error']*u.Mpc)

In [ ]:
trgb = ascii.read(basedir/'data'/'literature distances'/'PHANGSDistancesJuly23.txt',format='csv',delimiter='&')

In [ ]:
catalogues = {}
for name in results['name']:
    filename = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    catalogues[name] = ascii.read(filename,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, PNLF, pnlf
from pnlf.plot.pnlf import plot_pnlf
from pnlf.auxiliary import mu_to_parsec

name = 'NGC1385'

param = parameters[name]
cl = param['completeness_limit']
tbl = catalogues[name]
if False:
    data = tbl[((tbl['type']=='PN') | (tbl['SNRorPN']=='True')) & (tbl['exclude']==0)]['mOIII']
    err = tbl[((tbl['type']=='PN') | (tbl['SNRorPN']=='True')) & (tbl['exclude']==0)]['dmOIII']
else:
    data = tbl[(tbl['type']=='PN') & (tbl['exclude']==0) & (tbl['v_SIGMA']<1000) ]['mOIII']
    err  = tbl[(tbl['type']=='PN') & (tbl['exclude']==0)]['dmOIII']

fitter = MaximumLikelihood1D(pnlf,data[data<cl],mhigh=cl,Mmax=-4.47)
mu,mu_p,mu_m = fitter([28])
mu_p = np.sqrt(mu_p**2+np.nanmean(err)**2+dPSF**2)
mu_m = np.sqrt(mu_m**2+np.nanmean(err)**2+dPSF**2)
d,(dp,dm)=mu_to_parsec(mu,[mu_p,mu_m])
print('{:.2f} + {:.2f} - {:.2f}'.format(d,dp,dm))
print('{:.2f} + {:.2f} - {:.2f}'.format(mu,mu_p,mu_m))

#Plot PNLF
axes = plot_pnlf(data,
                 mu,
                 cl,
                 binsize=param['binsize'],
                 #mhigh=29,
                 filename=None,
                 color=tab10[0])


In [ ]:
tmp = tbl[(tbl['type']=='PN') & (tbl['mOIII']<29) & (tbl['exclude']==0)]
tmp.sort('mOIII')

plt.scatter(tmp['mOIII'],tmp['v_SIGMA'])
plt.axvline(np.min(tmp['mOIII'])+1)
plt.show()

In [ ]:
from pnlf.io import ReadLineMaps
galaxy = ReadLineMaps(data_raw,name,**parameters[name])


In [ ]:
from pnlf.plot.plot import plot_sky_with_detected_stars
tmp = tbl[tbl['exclude']==1]
positions = np.transpose((tmp['x'], tmp['y']))

plot_sky_with_detected_stars(data=galaxy.OIII5006_DAP,
                             wcs=galaxy.wcs,
                             positions=positions
                             )

## Match catalogue

In [ ]:
name = 'NGC0628'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])


In [ ]:
RA=24.1888
DEC=15.7968

coord = SkyCoord(RA*u.degree,DEC*u.degree)

sep = coord.separation(catalogue['SkyCoord'])

catalogue[np.argmin(sep)][['id','type']]